# Reproducing in-text result 3

This result starts with the doctor utilisation from Figure 2A which is over 100% when patient load is 170 (IAT 3) and appointment times are 5 minutes (mean 5, SD 1, boundary 2).

The variant introduced is to reduce the doctor intervention during delivery so that:

* 50% delivery cases: no doctor
* 30% delivery cases: doctor but only one-third of typical intervention
* 20% delivery cases: doctor and full intervention

## Set up

In [1]:
# To run model
import PHC
from reproduction_helpers import process_results

# To import results and produce figures
import xlrd
import pandas as pd
import os
import matplotlib.pyplot as plt
import numpy as np

# To speed up run time
from multiprocessing import Pool

In [2]:
# Paths to save image files to
output_folder = 'outputs'
save_path = os.path.join(output_folder, 'intext3.csv')

## Run model

In [3]:
# Parameters used in both models
base_model = {
    'OPD_iat': 3,
    'rep_file': 'arr170',
    'mean': 5,
    'sd': 1,
    'consult_boundary_1': 2,
    'consult_boundary_2': 2
}

# Model variants
variants = [
    {
        'doctor_delivery_scenario': False,
        'rep_file': 'in3_normal_delivery.xls'
    },
    {
        'doctor_delivery_scenario': True,
        'rep_file': 'in3_reduced_delivery.xls'
    },
]

In [4]:
# Combine dictionaries
dict_list = []
for var in variants:
    dict_list.append({**base_model, **var})

# Append 's_' to all items
for i, d in enumerate(dict_list):
    dict_list[i] = {f's_{k}': v for k, v in d.items()}

# View dictionaries
dict_list

[{'s_OPD_iat': 3,
  's_rep_file': 'in3_normal_delivery.xls',
  's_mean': 5,
  's_sd': 1,
  's_consult_boundary_1': 2,
  's_consult_boundary_2': 2,
  's_doctor_delivery_scenario': False},
 {'s_OPD_iat': 3,
  's_rep_file': 'in3_reduced_delivery.xls',
  's_mean': 5,
  's_sd': 1,
  's_consult_boundary_1': 2,
  's_consult_boundary_2': 2,
  's_doctor_delivery_scenario': True}]

In [5]:
# Wrapper function to allow input of dictionary with pool
def wrapper(d):
    return PHC.main(**d)

# Create a process pool that uses all CPUs
with Pool() as pool:
    # Run PHC.main() using each of inputs from config
    pool.map(wrapper, dict_list)

 No of replications done 0
 No of replications done 0
 No of replications done 1
 No of replications done 1
 No of replications done 2
 No of replications done 2
 No of replications done 3
 No of replications done 3
 No of replications done 4
 No of replications done 4
 No of replications done 5
 No of replications done 5
 No of replications done 6
 No of replications done 6
 No of replications done 7
 No of replications done 7
 No of replications done 8
 No of replications done 8
 No of replications done 9
 No of replications done 9


## Process results

In [6]:
data = process_results([i['s_rep_file'] for i in dict_list], xls=True)

In [7]:
def get_utilisation(index_name, new_name):
    '''
    Creates dataframe with columns for utilisation (for a given row) before and
    after scenario change.

    Parameters:
    -----------
    index_name : string
        Name of row with utilisation data
    new_name : string
        To rename the utilisation row (e.g. doc occ -> doctor utilisation)

    Returns:
    --------
    util : pandas DataFrame
        Dataframe with three columns for utilisation before and after admin,
        and change in utilisation.
    '''
    # Get utilisation
    util = round(pd.DataFrame(data.loc[index_name]).T, 3)

    # Find change
    util['Change'] = util['in3_reduced_delivery'] - util['in3_normal_delivery']

    # Rename index
    util = util.rename_axis('Output')
    # Rename row
    util = util.rename({index_name: new_name})
    # Rename columns for clarity for readers
    util = util.rename({
        'in3_normal_delivery': 'Normal delivery',
        'in3_reduced_delivery': 'Reduced doctor intervention'}, axis=1)

    return util

In [8]:
# Get results for doctor utilisation and staff nurse utilisation
util = pd.concat([
    get_utilisation('doc occ', 'Doctor utilisation'),
    get_utilisation('staff nurse occ', 'Staff nurse utilisation')])

# Display results
display(util)

# Save results
util.to_csv(save_path, index=True)

,Normal delivery,Reduced doctor intervention,Change
Output,,,
Doctor utilisation,1.144,1.129,-0.015
Staff nurse utilisation,0.326,0.320,-0.006


In [9]:
data['Change'] = data['in3_reduced_delivery'] - data['in3_normal_delivery']
data

,in3_normal_delivery,in3_reduced_delivery,Change
OPD patients,44025.500000,43964.300000,-61.200000
IPD patients,183.000000,181.700000,-1.300000
ANC patients,365.800000,354.600000,-11.200000
Del patients,372.300000,358.300000,-14.000000
OPD Q wt,7.026234,6.639372,-0.386861
Pharmacy Q wt,1.279974,1.284276,0.004302
Lab Q wt,3.152327,3.114581,-0.037746
doc occ,1.143773,1.129133,-0.014640
Lab patient list,253984.800000,253251.900000,-732.900000
OPD q len,6.964342,6.618311,-0.346031
